In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adamax
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout
from tensorflow.keras.models import Sequential
import os, os.path
from tensorflow import keras
import matplotlib.pyplot as plt
import PIL
from tensorflow.keras.layers import Dropout
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from os import walk
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing import image
import pathlib
import glob
import splitfolders
import random
import shutil
%matplotlib inline
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import to_categorical
import tensorboard
import SimpleITK as sitk
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Concatenate, Input
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix, f1_score
import seaborn as sns
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from sklearn.model_selection import train_test_split, StratifiedKFold
import cv2 
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
csv_file_path = 'Sarcoidosis AI (287 patients to 09-2023) - clusters.csv'
data_ = pd.read_csv(csv_file_path)

In [ ]:
features = data_[['Study']]  # Seleziona solo la colonna 'Study'
labels = data_['Systemic Sarcoidosis (POSITIVE = 1)']  # Utilizza solo la label per la sarcoidosi sistemica
print(labels)

In [ ]:
folder_path = 'sarcoidosis_MIPs_(npy)'
# List to save data loaded from each numpy file
data_list = []
new_height=400 
new_width=200
# Scan folder and load each numpy file
for filename in os.listdir(folder_path):
    if filename.endswith('.npy'):
        file_path = os.path.join(folder_path, filename)
        data = np.load(file_path)
        image=[]
        slice1=data[0]
        #slice2=data[3]
        #slice3=data[5]
        #slice4=data[7]

        # Crop before resizing        
        resized_slice1 = cv2.resize(slice1, (new_width, new_height))
        #resized_slice2 = cv2.resize(slice2, (new_width, new_height))
        #resized_slice3 = cv2.resize(slice3, (new_width, new_height))
        #resized_slice4 = cv2.resize(slice4, (new_width, new_height))

        # Crop to the resized image
        cropped_slice1 = resized_slice1[60:300, 20:200]
        #cropped_slice2 = resized_slice2[50:370, :200]
        #cropped_slice3 = resized_slice3[50:370, :200]
        #cropped_slice4 = resized_slice4[50:370, :200]

            #cropped_slice=cropped_slice / 255.0
        #image.append(cropped_slice1)
        #image.append(cropped_slice2)
        #image.append(cropped_slice3)
        #image.append(cropped_slice4)
        study_id = int(filename.split('_')[0])
       # Initialize a list to store matching labels
        matching_labels = []

        # ID patient

        # Find the corresponding patient label in the DataFrame
        matching_label = data_[data_['Study'] == study_id]['Systemic Sarcoidosis (POSITIVE = 1)'].values
        if len(matching_label) > 0:
            matching_labels.append(matching_label[0])
        else:
            matching_labels.append(None)  
            
    	# Add the corresponding patient image and label to the data list
        data_list.append((np.array(cropped_slice1), matching_labels))

In [ ]:
for item in data_list:
    image_array, label = item  
print("Image shape:", image_array.shape)  
print("Label:", label) 
print() 

In [ ]:
# Inizializza le liste per le features e le label
features_list = []
labels_list = []

# Estrai le features e le label da data_list
for item in data_list:
    features, label = item
    features_list.append(features)
    labels_list.append(label)

# Converti le liste in array numpy
features_array = np.array(features_list)
labels_array = np.array(labels_list)

In [ ]:
print(np.shape(features_array))
plt.imshow(features_array[9])

In [ ]:
print(np.shape(features_array))
image=(features_array[98])

In [ ]:
def calculate_grad_cam(model, layer_name, image):
    # Create a graph that outputs target convolution and output
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(layer_name).output, model.output])

    # Get the score for target class
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(np.array([image]))
        loss = predictions[0]

    # Extract filters and gradients
    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]
    # Average gradients spatially
    weights = tf.reduce_mean(grads, axis=(0, 1))

    # Build a ponderated map of filters according to gradients importance
    cam = np.ones(output.shape[0:2], dtype=np.float32)

    for index, w in enumerate(weights):
        cam += w * output[:, :, index]

    # Heatmap visualization
    cam = cv2.resize(cam.numpy(), (180, 240))
    cam = np.maximum(cam, 0)
    heatmap = (cam - cam.min()) / (cam.max() - cam.min())

    cam = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)

    return cam

In [ ]:

# Definition of the number of folds for cross-validation
n_splits =5
input_shape = (240, 180, 3)  

accuracy_scores = []
f1_scores = []

for i in range(n_splits):
    print(f"Iterazione {i+1}:")
    
    X_train, X_test, y_train, y_test = train_test_split(features_array, labels_array, test_size=0.3, random_state=i, stratify=labels_array)

    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.30, random_state=i, stratify=y_test)

    # Data augmentation
    def saturate_image(image):
        hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        hsv_image[..., 1] *= np.random.uniform(0.5, 1.5)  
        return cv2.cvtColor(hsv_image, cv2.COLOR_HSV2RGB)
    def adjust_contrast(image, factor):
        return tf.image.adjust_contrast(image, factor)

    def custom_contrast(image):
        factor = tf.random.uniform([], 0.4, 1)  
        return adjust_contrast(image, factor)

    train_datagen = ImageDataGenerator(
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=custom_contrast
        #vertical_flip=True,
        #brightness_range=[0.2, 0.5]
        
        
    )

   # Creation of the complete model
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    # Imposta i layer come non allenabili
    for layer in vgg16.layers:
        layer.trainable = False

   # Create a new sequential pattern
    x = Flatten()(vgg16.output)
    x = Dense(512, activation='relu' ,kernel_regularizer=l1(0.001))(x)
    x = Dropout(0.3)(x)  # Aggiungi un layer di Dropout con una probabilità del 50%
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=vgg16.input, outputs=predictions)
    model.summary()

    X_train = X_train / 255
    X_val = X_val / 255
    X_test = X_test / 255

    train_generator = train_datagen.flow(X_train, y_train, batch_size=32)

    model.compile(optimizer=Adamax(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(train_generator, epochs=50, validation_data=(X_val, y_val))

    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print("Test Accuracy:", test_accuracy)

    test_predictions = model.predict(X_test)
    test_predictions = (test_predictions > 0.5).astype(int)
    f1 = f1_score(y_test, test_predictions)
    print("F1-score:", f1)

    accuracy_scores.append(test_accuracy)
    f1_scores.append(f1)

    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    # GRAD-CAM
    conv_layer = 'block5_conv3'
    num_images_to_display = min(len(X_test), 30)  
    rows = int(np.ceil(num_images_to_display / 5))  

    plt.figure(figsize=(15, 3 * rows))  
    for i in range(num_images_to_display):
        cam = calculate_grad_cam(model, conv_layer, X_test[i])
        plt.subplot(rows, 5, i+1)
        plt.imshow(X_test[i])
        plt.imshow(cam, alpha=0.5, cmap='jet')
        title = "Correct" if test_predictions[i] == y_test[i] else "Incorrect"
        title += f" (Class {y_test[i]})"  
        plt.title(title)
        plt.axis('off')
    plt.tight_layout()  
    plt.show()

    # Confusion Matrix
    conf_matrix = confusion_matrix(y_test, test_predictions)
    print("Confusion Matrix:")
    print(conf_matrix)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')
    plt.show()

print("Average Accuracy:", np.median(accuracy_scores))
print("Average F1-score:", np.median(f1_scores))
print("Standard Deviation of Accuracy:", np.std(accuracy_scores))
print("Standard Deviation of F1-score:", np.std(f1_scores))

In [ ]:
print("Average Accuracy:", np.mean(accuracy_scores))
print("Average F1-score:", np.mean(f1_scores))
print("Standard Deviation of Accuracy:", np.std(accuracy_scores))
print("Standard Deviation of F1-score:", np.std(f1_scores))

In [ ]:
print(np.percentile(accuracy_scores,25))
print(np.percentile(f1_scores,25))

In [ ]:
print(np.percentile(accuracy_scores,75))
print(np.percentile(f1_scores,75))